<a href="https://colab.research.google.com/github/dhorvath/AI-Stuff/blob/main/LlamaIndex_ReAct_Agent_with_Query_Engine_(RAG)_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup an agent powered by the ReAct loop for financial analysis.
Following the instructions here to test:
https://docs.llamaindex.ai/en/stable/examples/agent/react_agent_with_query_engine/

In [1]:
!pip install llama-index-readers-file
!pip install llama-index-llms-openai
!pip install llama-index-embeddings-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 7.0 MB/s eta 

In [16]:
!pip install textwrap3

In [2]:
# Imports
from openai import OpenAI
from google.colab import userdata
from google.colab import files
import os

# API
open_ai_key = userdata.get('open_ai_key')
client = OpenAI(api_key=open_ai_key)

os.environ["OPENAI_API_KEY"] = open_ai_key

In [17]:
# Helper
import textwrap3
def wrap_print(long_text):
  print('\n'.join(textwrap3.wrap(long_text)))

In [3]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [4]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [5]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-08-26 20:01:59--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  7.55MB/s    in 0.2s    

2024-08-26 20:02:00 (7.55 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-08-26 20:02:00--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connec

In [6]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [7]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [8]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

# Setup ReAct Agent

In [9]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [12]:
context = "You are a stock market sorcerer who is an expert on the companies Lyft and Uber. You will answer questions about Uber and Lyft as in the persona of a sorcerer and veteran stock market investor."

llm = OpenAI(model="gpt-3.5-turbo")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

In [20]:
response = agent.chat("What was Lyft's revenue growth in 2021?")
wrap_print(str(response))

> Running step 3ab4da3c-b434-44b3-87ec-3084f9195039. Step input: What was Lyft's revenue growth in 2021?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: Lyft's revenue increased by 36% in 2021 compared to the prior year.
> Running step b9063441-237b-40d5-b16f-9e9968434edb. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Lyft's revenue grew by 36% in 2021 compared to the previous year.
Lyft's revenue grew by 36% in 2021 compared to the previous year.


In [19]:
response = agent.chat(
    "Compare and contrast the revenue growth of Uber and Lyft in 2021, then"
    " give an analysis"
)
wrap_print(str(response))

> Running step 81f6d5de-0d18-43d0-8790-c7fbc0318c9f. Step input: Compare and contrast the revenue growth of Uber and Lyft in 2021, then give an analysis
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: In 2021, Uber's revenue growth was higher at 57% compared to Lyft's revenue growth of 36%. This indicates that Uber experienced a stronger growth in revenue during that year. The higher revenue growth for Uber could be attributed to various factors such as market expansion, diversification of services, and potentially more aggressive marketing strategies. On the other hand, Lyft's slightly lower revenue growth may be influenced by factors like market competition, regulatory challenges, and specific operational decisions made by the company.
In 2021, Uber's revenue growth was higher at 57% compared to Lyft's
revenue growth of 36%. This indicates that Uber experienced a stronger
growth in revenue during that year. The higher revenue growth 

In [18]:
response = agent.chat(
    "Can you tell me about the risk factors of the company with the higher"
    " revenue?"
)
wrap_print(str(response))

> Running step 7640231e-0b56-4b81-97bd-4a9b2e845200. Step input: Can you tell me about the risk factors of the company with the higher revenue?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: uber_10k
Action Input: {'input': 'Please provide information on the risk factors of the company with higher revenue in 2021.'}
Observation: The risk factors of the company with higher revenue in 2021 include the potential inability to accurately forecast operating results due to challenging expense level estimates, fixed expenses that may not be adjusted quickly if revenue falls short, the risk of sustained losses impacting investor value, the possibility of not achieving profitability if growth slows significantly, and the expectation of continued slowing in Gross Bookings and revenue growth rates. Additionally, factors such as the duration and severity of the COVID-19 pandemic, the need to grow supply and demand on the platform,